# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [20]:
import pandas as pd
import pymongo

In [88]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['youtube']

In [89]:
from datetime import datetime
df_youtube = pd.read_csv("./data/USvideos.csv", parse_dates=['date'])
df_youtube['date'] = df_youtube['date'] + '.2020'
df_youtube['date'] = df_youtube['date'].apply(lambda x : datetime.strptime(x, "%d.%M.%Y"))
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,2020-01-13 00:09:00
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,2020-01-13 00:09:00
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,2020-01-13 00:09:00
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,2020-01-13 00:09:00
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,2020-01-13 00:09:00


## Question 0

### Nettoyer les données

In [90]:
import json
with open("./data/US_category_id.json",'r') as load_f:
    df_json = json.load(load_f)
    for element in df_json['items']:
        element['channelID'] = element['snippet']['channelId']
        element['title']=element['snippet']['title']
        element['assignable']=element['snippet']['assignable']
        element.pop('snippet')
    df_json = pd.DataFrame.from_dict(df_json['items'])
df_json[['category_id']] = df_json[['id']].astype(int)

df_json.head()

,kind,etag,id,channelID,title,assignable,category_id
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True,1
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True,2
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True,10
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True,15
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True,17


In [91]:

df_youtube = df_youtube.rename(columns={"id":"category_id"})

In [92]:
df_youtube.dtypes


video_id                  object
title                     object
channel_title             object
category_id                int64
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_total              int64
thumbnail_link            object
date              datetime64[ns]
dtype: object

In [93]:
df_youtube = pd.merge(df_youtube, df_json, left_on='category_id', right_on='category_id')


In [94]:
df_youtube.rename(columns={"video_id" : "_id"})

,_id,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,id,channelID,title_y,assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,2020-01-13 00:09:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",24,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,2020-01-13 00:09:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",24,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,America's Got Talent 2017|america's got talent...,473691,14740,415,1696,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,2020-01-13 00:09:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",24,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,panda|what should we eat|buzzfeed|comedy|boyfr...,282858,14870,300,1398,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,2020-01-13 00:09:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",24,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,2020-01-13 00:09:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",24,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7987,ThPzNokG5uE,Japanese Candy Art,INSIDER,19,Shinobu Ichiyanagi|Japanese|Candy|Art|Amezaiku...,262878,7594,62,663,https://i.ytimg.com/vi/ThPzNokG5uE/default.jpg,2020-01-19 00:01:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",19,UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7988,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,383835,10457,275,1948,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,2020-01-19 00:01:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",19,UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7989,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,396811,10633,279,1552,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,2020-01-20 00:01:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",19,UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7990,5mdk5oz6l3w,Pixel 2: Squeeze to access features (CNET News),CNET,43,Google|Pixel 2|Squeeze|Google Assistant|Phones...,8259,111,8,37,https://i.ytimg.com/vi/5mdk5oz6l3w/default.jpg,2020-01-06 00:01:00,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/0n9MJVCDLpA8q7aiG...",43,UCBR8-60-B28hp2BmDPdntcQ,Shows,False


In [95]:
df_youtube.columns

Index(['video_id', 'title_x', 'channel_title', 'category_id', 'tags', 'views',
       'likes', 'dislikes', 'comment_total', 'thumbnail_link', 'date', 'kind',
       'etag', 'id', 'channelID', 'title_y', 'assignable'],
      dtype='object')

### Importer les données

In [96]:
documents = df_youtube.to_dict(orient = 'records')

In [97]:
collection.insert_many(documents)

In [98]:
collection.find_one()

{'_id': ObjectId('5fc92a85f390b0bbb28d8926'),
 'video_id': 'XpVt6Z1Gjjo',
 'title_x': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': datetime.datetime(2020, 1, 13, 0, 9),
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"',
 'id': '24',
 'channelID': 'UCBR8-60-B28hp2BmDPdntcQ',
 'title_y': 'Entertainment',
 'assignable': True}

## Question 1  

In [99]:
cursor = collection.find({"channel_title":"Apple"})

In [100]:
for document in cursor :
    print('-------')
    print(document)

-------
{'_id': ObjectId('5fc92a85f390b0bbb28d8f67'), 'video_id': 'K4wEI5zhHB0', 'title_x': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 7860119, 'likes': 185853, 'dislikes': 26679, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': datetime.datetime(2020, 1, 13, 0, 9), 'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"', 'id': '28', 'channelID': 'UCBR8-60-B28hp2BmDPdntcQ', 'title_y': 'Science & Technology', 'assignable': True}
-------
{'_id': ObjectId('5fc92a85f390b0bbb28d8f77'), 'video_id': 'K4wEI5zhHB0', 'title_x': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|anim

## Question 2

In [101]:
count= collection.aggregate([{"$group" : {"_id" : "$category_id"}}])
len(list(count))

16

## Question 3

In [102]:
all_doc = collection.find({})


In [103]:
for document in all_doc:
    collection.update_one(document, {"$set": {"tags" : document["tags"].split('|')}})


In [108]:
next(collection.find({}))

{'_id': ObjectId('5fc92a85f390b0bbb28d8926'),
 'video_id': 'XpVt6Z1Gjjo',
 'title_x': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': datetime.datetime(2020, 1, 13, 0, 9),
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2B

## Question 4

In [ ]:
max_views = collection.find({}).sort([("views", -1)])
next(max_views)

## Question 5

In [ ]:
average_views_category = collection.aggregate([{"$group" : {"_id" : "$category_id", "views" : {"$avg" : "$views"}}}])
list(average_views_category).count()

## Question 6 

In [ ]:
max_avg_like_channel = collection.aggregate([{"$group" : {"_id" : "$channel_title", "likes" : {"$avg" : "$likes"}}},{
                                            "$sort": {"likes" : -1}}])
list(max_avg_like_channel)